In [1]:
import pandas as pd
import re


In [2]:
contacts_df = pd.read_csv('/Users/michaelbarski/Desktop/personal_projects/contact-company-matching-poc/making_new_data/us-500.csv')

display(contacts_df)

,first_name,last_name,company_name,address,city,county,state,zip,phone1,phone2,email,web
0,James,Butt,"Benton, John B Jr",6649 N Blue Gum St,New Orleans,Orleans,LA,70116,504-621-8927,504-845-1427,jbutt@gmail.com,http://www.bentonjohnbjr.com
1,Josephine,Darakjy,"Chanay, Jeffrey A Esq",4 B Blue Ridge Blvd,Brighton,Livingston,MI,48116,810-292-9388,810-374-9840,josephine_darakjy@darakjy.org,http://www.chanayjeffreyaesq.com
2,Art,Venere,"Chemel, James L Cpa",8 W Cerritos Ave #54,Bridgeport,Gloucester,NJ,8014,856-636-8749,856-264-4130,art@venere.org,http://www.chemeljameslcpa.com
3,Lenna,Paprocki,Feltz Printing Service,639 Main St,Anchorage,Anchorage,AK,99501,907-385-4412,907-921-2010,lpaprocki@hotmail.com,http://www.feltzprintingservice.com
4,Donette,Foller,Printing Dimensions,34 Center St,Hamilton,Butler,OH,45011,513-570-1893,513-549-4561,donette.foller@cox.net,http://www.printingdimensions.com
...,...,...,...,...,...,...,...,...,...,...,...,...
495,Brittni,Gillaspie,Inner Label,67 Rv Cent,Boise,Ada,ID,83709,208-709-1235,208-206-9848,bgillaspie@gillaspie.com,http://www.innerlabel.com
496,Raylene,Kampa,Hermar Inc,2 Sw Nyberg Rd,Elkhart,Elkhart,IN,46514,574-499-1454,574-330-1884,rkampa@kampa.org,http://www.hermarinc.com
497,Flo,Bookamer,Simonton Howe & Schneider Pc,89992 E 15th St,Alliance,Box Butte,NE,69301,308-726-2182,308-250-6987,flo.bookamer@cox.net,http://www.simontonhoweschneiderpc.com
498,Jani,Biddy,Warehouse Office & Paper Prod,61556 W 20th Ave,Seattle,King,WA,98104,206-711-6498,206-395-6284,jbiddy@yahoo.com,http://www.warehouseofficepaperprod.com


In [3]:
contacts_df['company_name'] = contacts_df['company_name'].str.lower()

company_names = list(contacts_df.company_name)

company_names_with_suffixes = []
for i in company_names:
    if i.endswith('inc') or i.endswith('ltd') or i.endswith('cpa') or i.endswith('esq') or i.endswith(' co') or i.endswith(' corp'):
        company_names_with_suffixes.append(i)

# print(company_names_with_suffixes)



if the company name is in the suffixes list, take that name and remove the suffix. Maybe it would be easier to split this operation up into smaller pieces. 


In [4]:
reformatted_company_names = []
for name in company_names:
    if name in company_names_with_suffixes and (
        name.endswith(' inc') or
        name.endswith(' ltd') or 
        name.endswith(' cpa') or 
        name.endswith(' esq')):

        reformatted_company_names.append(name[:-4])

    elif name in company_names_with_suffixes and name.endswith(' co'):
        reformatted_company_names.append(name[:-3])
    elif name in company_names_with_suffixes and name.endswith(' corp'):
        reformatted_company_names.append(name[:-5])
    else:
        reformatted_company_names.append(name)

reformatted_company_names = pd.Series(reformatted_company_names)

below I'm creating a reformatted url series and creating another series that contains subdomains

we'll attempt to make urls in the following way:
- we'll make a copy of reformatted company_names
- we'll get rid of all of the white space and all of the special characters (, . ) 
- then we'll all '.com' to the end and replace the web column with contacts_df['company_url'] = better_urls
- last we'll have to think a little bit about integration with the downstream code; we added subdomains and spelling errors to each url.

In [5]:
better_urls = reformatted_company_names.copy()
pattern = re.compile(r'\W|\s')

better_urls = better_urls.str.replace(pattern, '')
better_urls = better_urls.astype(str) + '.com'

contacts_df['company_url'] = better_urls


In [6]:
urls = contacts_df.company_url
adding_subdomains = urls.sample(20)

with_subdomains = []
for i,value in enumerate(adding_subdomains):
    if i%2:
        value = f"support.{value}"
        with_subdomains.append(value)
    else:
        value = f"store.{value}"
        with_subdomains.append(value)


print(with_subdomains)

['store.stanleyrichardl.com', 'support.merrilllynch.com', 'store.changcarolyn.com', 'support.germerandgertzllp.com', 'store.unitedoilco.com', 'support.maccanidelp.com', 'store.kbor1600am.com', 'support.rodewayinn.com', 'store.delcharroapartments.com', 'support.nagledanielj.com', 'store.matriccianialbertjjr.com', 'support.upullit.com', 'store.olympicgraphicarts.com', 'support.breslereitelframggllry.com', 'store.rolandashcroft.com', 'support.centerlineengineering.com', 'store.mitsumielectronics.com', 'support.garrisonind.com', 'store.barkannealj.com', 'support.feinerbros.com']


Here I'm replacing the version of the url that does not have the subdomain with the version that does

In [7]:
altered_urls = urls.copy()
for i in altered_urls:
    for j in with_subdomains:
        if i in j:
            altered_urls.replace(i,j, inplace=True)



adding typos to make it so we can't match on url alone

In [8]:
typo_urls = altered_urls.sample(40)
typo_urls = [i[1:] for i in typo_urls]

for i in altered_urls:
    for j in typo_urls:
        if j in i:
            altered_urls.replace(i,j, inplace=True)

print(typo_urls)

['arrisonpaule.com', 'lenridgemanormobilehomepk.com', 'eutelschiescompany.com', 'upport.garrisonind.com', 'orlongassociates.com', 'iconcorporation.com', 'urrowsjonh.com', 'oyalpontiacolds.com', 'eramictilesales.com', 'ioruccifoodsusa.com', 'disonsupplyequipment.com', 'usinesssystemsofwis.com', 'entraldiecastingmfgco.com', 'ampbellroberta.com', 'yeeproductions.com', 'ellskravitzschnitzer.com', 'allacechurchassoc.com', 'auiresearchtechnologypk.com', 'heatonplasticproducts.com', 'ricebusinessservices.com', 'ranz.com', 'linabal.com', 'erkroberte.com', 'umminssouthernplains.com', 'hanayjeffreya.com', 'terocompany.com', 'hompsonfabricating.com', 'bsbusinessfinance.com', 'loralexpressions.com', 'heatleytruckingcompany.com', 'orestfirelaboratory.com', 'hankedwardl.com', 'wiggsabramsblanchard.com', 'enbrookmyron.com', 'tuartjagins.com', 'oloveroberta.com', 'tore.changcarolyn.com', 'lpenlite.com', 'eneralfoamcorporation.com', 'cglynnblisspc.com']


adding random revenues and tech for each company

In [ ]:
import random

random_revenues = pd.Series([random.randint(5000000,1000000000) for i in urls])

tech_list = ['shopify', 'amazon pay', 'salesforce', 'zendesk', 'bigcommerce', 'hubspot', 'marketo', 'google analytics']

random_techs = pd.Series([random.choice(tech_list) for i in urls])

Here I'll start making my new dataframes using some pieces of the old ones

In [ ]:
exported_contacts_df = contacts_df.drop(['phone1', 'phone2', 'web'], axis=1)
exported_contacts_df['company_name'] = reformatted_company_names

company_data_df = pd.DataFrame({'company_name':contacts_df.company_name, 'company_url':altered_urls, 'revenue':random_revenues, 'current_tech_use':random_techs})



make a new list of company names and company new urls from the exported_contacts:
change these lists to only be 100 in length (instead of 500) -- could also make a new df
then repeat each name 5 times or append or add the list to itself to get 500 again (with each company name and company url repeated 5 times)

In [ ]:
fixing_names_df = exported_contacts_df[['company_name', 'company_url']]
fixing_names_df = fixing_names_df.iloc[:100,:]
redoing_companies_to_contacts = fixing_names_df.loc[fixing_names_df.index.repeat(5)].reset_index(drop=True)

new_exported_contacts_df = exported_contacts_df.copy()

new_exported_contacts_df['company_name'] = redoing_companies_to_contacts.company_name
new_exported_contacts_df['company_url'] = redoing_companies_to_contacts.company_url

new_exported_contacts_df.drop_duplicates(['email'], inplace=True)

In [ ]:
new_exported_contacts_df.to_csv('new_exported_contacts.csv', index=False)
company_data_df.to_csv('company_data.csv', index=False)